In [51]:
import json 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from hypyp import stats 
'''
Comparing inter-connectivity to random signal
'''

f = open('cross_freq_results.json')
# f = open('results_plv.json')

# returns JSON object as 
# a dictionary
results = json.load(f)
  
# Iterating through the json
# list
fp1 = []
sf1 = []
fp2 = []
sf2 = []
reunion = []

all_stages = [fp1, sf1, fp2, sf2, reunion]
for part, data in results.items():
    for stage, bands in data.items():
        stage = stage[1]
        all_stages[int(stage)-1].append(bands)

fp1_theta = []
fp1_alpha = []
sf1_theta = []
sf1_alpha = []
fp2_theta = []
fp2_alpha = []
sf2_theta = []
sf2_alpha = []
ru_theta = []
ru_alpha = []
for part1, part2, part3, part4, part5 in zip(fp1, sf1, fp2, sf2, reunion):
    fp1_theta.append(np.mean(part1['theta']))
    fp1_alpha.append(np.mean(part1['alpha']))
    sf1_theta.append(np.mean(part2['theta']))
    sf1_alpha.append(np.mean(part2['alpha']))
    fp2_theta.append(np.mean(part3['theta']))
    fp2_alpha.append(np.mean(part3['alpha']))
    sf2_theta.append(np.mean(part4['theta']))
    sf2_alpha.append(np.mean(part4['alpha']))
    ru_theta.append(np.mean(part5['theta']))
    ru_alpha.append(np.mean(part5['alpha']))


data_theta = [fp1_theta, sf1_theta, fp2_theta, sf2_theta, ru_theta] 
data_alpha = [fp1_alpha, sf1_alpha, fp2_alpha, sf2_alpha, ru_alpha] 

f.close()

## Testing for Normal Distribution

In [52]:
import pandas as pd
import pingouin as pg
from statsmodels.stats.anova import AnovaRM
from scipy.stats import shapiro

# Load data
df = pd.DataFrame({"FreePlay1": fp1_theta,
                  "StillFace1": sf1_theta,
                  "FreePlay2": fp2_theta,
                  "StillFace2": sf2_theta,
                  "Reunion": ru_theta})


# Create a multi-indexed dataframe
df_melted_theta = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['FreePlay1', 'StillFace1', 'FreePlay2', 'StillFace2', 'Reunion'])

# Rename columns
df_melted_theta.columns = ['id', 'condition', 'value']


# grouped = df_melted.groupby(["condition"])

data = np.random.normal(0, 1, 100)
stat, p = shapiro(df_melted_theta["value"])

print("Test statistic:", stat)
print("p-value:", p)

Test statistic: 0.8499273657798767
p-value: 6.362323428515948e-12


In [54]:
import pandas as pd
import pingouin as pg
from scipy.stats import shapiro

# Load data
df = pd.DataFrame({"FreePlay1": fp1_alpha,
                  "StillFace1": sf1_alpha,
                  "FreePlay2": fp2_alpha,
                  "StillFace2": sf2_alpha,
                  "Reunion": ru_alpha})


# Create a multi-indexed dataframe
df_melted_alpha = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['FreePlay1', 'StillFace1', 'FreePlay2', 'StillFace2', 'Reunion'])

# Rename columns
df_melted_alpha.columns = ['id', 'condition', 'value']

# grouped = df_melted.groupby(["condition"])

data = np.random.normal(0, 1, 100)
stat, p = shapiro(df_melted_alpha["value"])

print("Test statistic:", stat)
print("p-value:", p)


Test statistic: 0.9676814079284668
p-value: 0.0005415018531493843


## Fisher Z-transform

In [55]:
# Perform the Fisher z-transform
z_fp1_theta = np.arctanh(fp1_theta)
z_sf1_theta = np.arctanh(sf1_theta)
z_fp2_theta = np.arctanh(fp2_theta)
z_sf2_theta = np.arctanh(sf2_theta)
z_ru_theta = np.arctanh(ru_theta)

# Load data
df = pd.DataFrame({"FreePlay1": z_fp1_theta,
                  "StillFace1": z_sf1_theta,
                  "FreePlay2": z_fp2_theta,
                  "StillFace2": z_sf2_theta,
                  "Reunion": z_ru_theta})


# Create a multi-indexed dataframe
df_melted = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['FreePlay1', 'StillFace1', 'FreePlay2', 'StillFace2', 'Reunion'])

# Rename columns
df_melted.columns = ['id', 'condition', 'value']

data = np.random.normal(0, 1, 100)
stat, p = shapiro(df_melted["value"])

print("Test statistic:", stat)
print("p-value:", p)

Test statistic: 0.8520227670669556
p-value: 7.938313201227842e-12


In [57]:
# Perform the Fisher z-transform
z_fp1_alpha = np.arctanh(fp1_alpha)
z_sf1_alpha = np.arctanh(sf1_alpha)
z_fp2_alpha = np.arctanh(fp2_alpha)
z_sf2_alpha = np.arctanh(sf2_alpha)
z_ru_alpha= np.arctanh(ru_alpha)

# Load data
df = pd.DataFrame({"FreePlay1": z_fp1_alpha,
                  "StillFace1": z_sf1_alpha,
                  "FreePlay2": z_fp2_alpha,
                  "StillFace2": z_sf2_alpha,
                  "Reunion": z_ru_alpha})


# Create a multi-indexed dataframe
df_melted = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['FreePlay1', 'StillFace1', 'FreePlay2', 'StillFace2', 'Reunion'])

# Rename columns
df_melted.columns = ['id', 'condition', 'value']

data = np.random.normal(0, 1, 100)
stat, p = shapiro(df_melted["value"])

print("Test statistic:", stat)
print("p-value:", p)

Test statistic: 0.9676247835159302
p-value: 0.0005339072085916996


## Friedman Test
The null hypothesis (H0): The mean for each population is equal.

The alternative hypothesis: (Ha): At least one population mean is different from the rest.

https://www.statology.org/friedman-test-python/

### theta

In [58]:
# Perform Friedman test
pg.friedman(data=df_melted_theta, dv='value', within='condition', subject = 'id')

,Source,W,ddof1,Q,p-unc
Friedman,condition,0.186678,4,25.388235,0.000042


In [59]:
#perform Friedman Test      
from scipy.stats import friedmanchisquare
friedmanchisquare(fp1_theta, sf1_theta, fp2_theta, sf2_theta, ru_theta)

FriedmanchisquareResult(statistic=25.388235294117635, pvalue=4.2028975896446456e-05)

### alpha

In [60]:
# Perform Friedman test
pg.friedman(data=df_melted_alpha, dv='value', within='condition', subject = 'id')

,Source,W,ddof1,Q,p-unc
Friedman,condition,0.046194,4,6.282353,0.179031


In [61]:
friedmanchisquare(fp1_alpha, sf1_alpha, fp2_alpha, sf2_alpha, ru_alpha)

FriedmanchisquareResult(statistic=6.282352941176441, pvalue=0.17903095565368235)